# Deep Q-Learning

Manual Implementation

---

**Deep Reainforcement Learning vs. Deep Q-Learning**

Deep Reinforcement Learning describes a reinforcement approach combines with Deep Learning approach.

Deep Q-Learning is a special subtopic of this. Here the reinforcement learning approach i set to the Q-Learning-approach (combined with deep learning)

---

In [ ]:
#!{sys.executable} -m pip install tensorflow==2.6.2

In [ ]:
%pip show tensorflow

**imports**

In [1]:
import numpy as np

import random
import sys
from collections import deque

import gym

In [2]:
# for network
from tensorflow.keras.models import Sequential, clone_model
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam

C:\Users\tobia\AppData\Roaming\Python\Python39\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
c:\Users\tobia\.conda\envs\ai\lib\site-packages\keras\utils\image_utils.py:36: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': pil_image.NEAREST,
c:\Users\tobia\.conda\envs\ai\lib\site-packages\keras\utils\image_utils.py:37: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  'bilinear': pil_image.BILINEAR,
c:\Users\tobia\.conda\envs\ai\lib\site-packages\keras\utils\image_utils.py:38: DeprecationWarning: BICUBIC is deprecated and will be removed 

**setup env**

In [3]:
env_name = "CartPole-v1"

env = gym.make(env_name)

C:\Users\tobia\AppData\Roaming\Python\Python39\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
c:\Users\tobia\.conda\envs\ai\lib\site-packages\gym\core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
c:\Users\tobia\.conda\envs\ai\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


try out

In [4]:
def tryout():
    env = gym.make(env_name)
    env.reset()

    for step in range(500):
        env.render()
        random_action = env.action_space.sample()
        obs, reward, done, info = env.step(random_action)
    env.close()

In [5]:
# tryout()

**Artificial Neural Network**

In [6]:
num_observations = env.observation_space.shape[0]
num_observations

4

In [7]:
num_actions = env.action_space.n
num_actions

2

In [8]:
# input -> obs -> 4 ===> 2 (Actions)
model = Sequential()

# (1, num)
model.add(Dense(16, input_shape=(1, num_observations)))
model.add(Activation('relu'))

model.add(Dense(32))
model.add(Activation('relu'))

model.add(Dense(num_actions))
model.add(Activation('linear'))

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1, 16)             80        
                                                                 
 activation (Activation)     (None, 1, 16)             0         
                                                                 
 dense_1 (Dense)             (None, 1, 32)             544       
                                                                 
 activation_1 (Activation)   (None, 1, 32)             0         
                                                                 
 dense_2 (Dense)             (None, 1, 2)              66        
                                                                 
 activation_2 (Activation)   (None, 1, 2)              0         
                                                                 
Total params: 690
Trainable params: 690
Non-trainable pa

In [10]:
# target-network
target_model = clone_model(model)

**hyperparameters**

In [11]:
EPOCHS = 1000

epsilon = 1.0
EPSILON_REDUCE = 0.995

# not alpha
LEARNING_RATE = 0.001
GAMMA = 0.95

BATCH_SIZE = 32

**functions**

In [12]:
def epsilon_greedy_action_selection(model, epsilon, observation):
    if np.random.random() > epsilon:
        prediction = model.predict(observation)    # at example: [0.4, 0.6]
        action = np.argmax(prediction)
    else:
        action = np.random.randint(0, env.action_space.n)
    return action

deque example:

In [13]:
deque_1 = deque(maxlen=3)
deque_1

deque([])

In [14]:
for i in range(5):
    deque_1.append(i)
    print(deque_1)

deque([0], maxlen=3)
deque([0, 1], maxlen=3)
deque([0, 1, 2], maxlen=3)
deque([1, 2, 3], maxlen=3)
deque([2, 3, 4], maxlen=3)


replay buffer:

In [15]:
replay_buffer = deque(maxlen=20000)
update_target_model = 10

In [16]:
# showcase
test_tuple = [(1,2,3), (4,5,6), (7,8,9)]
zipped_list = list(zip(*test_tuple))
a,b,c = zipped_list
a

(1, 4, 7)

In [17]:
def replay(replay_buffer, batch_size, model, target_model):
    if len(replay_buffer) < batch_size:
        return

    samples = random.sample(replay_buffer, batch_size)

    target_batch = []

    zipped_samples = list(zip(*samples))
    states, actions, rewards, new_states, dones = zipped_samples

    targets = target_model.predict(np.array(states))

    q_values = model.predict(np.array(new_states))

    for i in range(batch_size):
        q_value = max(q_values[i][0])
        target = targets[i].copy()
        if dones[i]:
            target[0][actions[i]] = rewards[i]
        else:
            target[0][actions[i]] = rewards[i] + q_value*GAMMA

        target_batch.append(target)

    model.fit(np.array(states), np.array(target_batch), epochs=1, verbose=0)

In [18]:
def update_model_handler(epoch, update_target_model, model, target):
    if epoch > 0 and epoch % update_target_model == 0:
        target_model.set_weights(model.get_weights())

**training**

In [19]:
model.compile(loss='mse', optimizer=(Adam(lr=LEARNING_RATE)))

c:\Users\tobia\.conda\envs\ai\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [20]:
best_so_far = 0

for epoch in range(EPOCHS):
    observation = env.reset()

    # (1, X) -> [a,b,c,d] -> 4 -> rehape (1,4)
    observation = observation.reshape([1,4])
    done = False
    points = 0

    while not done:
        try:
            action = epsilon_greedy_action_selection(model, epsilon, observation)
        except Exception as e:
            print(observation.shape)
            print(observation)
            print(epoch)
            print(e)
            sys.exit()
            #raise e

        next_observation, reward, done, info = env.step(action)
        next_observation = next_observation.reshape([1, 4])

        replay_buffer.append((observation, action, reward, next_observation, done))
        observation = next_observation
        points += 1

        replay(replay_buffer, BATCH_SIZE, model, target_model)

    epsilon *= EPSILON_REDUCE

    update_model_handler(epoch, update_target_model, model, target_model)

    if points > best_so_far:
        best_so_far = points
        
    if epoch % 25 == 0:
        print(f"Epoch:{epoch}    Points:{points}    eps:{epsilon}    BSF:{best_so_far}")

Epoch:0    Points:23    eps:0.995    BSF:23
1/1 [==============================] - 0s 11ms/step


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



(1, 4)
[[ 0.06033813  0.63496405 -0.13960798 -1.2264875 ]]
6
Graph execution error:

Detected at node 'sequential/dense/Tensordot/GatherV2_1' defined at (most recent call last):
    File "c:\Users\tobia\.conda\envs\ai\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\tobia\.conda\envs\ai\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\tobia\AppData\Roaming\Python\Python39\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "c:\Users\tobia\.conda\envs\ai\lib\site-packages\traitlets\config\application.py", line 976, in launch_instance
      app.start()
    File "C:\Users\tobia\AppData\Roaming\Python\Python39\site-packages\ipykernel\kernelapp.py", line 612, in start
      self.io_loop.start()
    File "C:\Users\tobia\AppData\Roaming\Python\Python39\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_f

TypeError: object of type 'NoneType' has no len()

**Using trained model**

In [ ]:
observation = env.reset()

for step in range(300):
    env.render()
    action = np.argmax(model.predict(observation.reshape([1,4])))
    observation, reward, done, info = env.step(action)

    if done:
        break
env.close()